## **Usage of Basic Pitch**
1. Inference on Basic Pitch
2. Conversion of MIDI to Tabs
3. Reading chords from MIDI
4. Saving output as MIDI file

In [ ]:
from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH
from mido import MidiFile, MidiTrack, Message, MetaMessage
import mido


### Inference on Basic Pitch

In [ ]:

model_output, midi_data, note_events = predict('c-chord_filtered.wav')


### Conversion of MIDI to Tabs

In [ ]:
def midi_pitch_to_guitar_tab(pitch):
    # Standard guitar tuning (from low to high)
    guitar_tuning = [40, 45, 50, 55, 59, 64]  # E2, A2, D3, G3, B3, E4


    best_string = 0
    best_fret = float('inf')

    for string, open_pitch in enumerate(guitar_tuning):
        if pitch >= open_pitch:
            fret = pitch - open_pitch
            if fret < best_fret:
                best_string = string
                best_fret = fret

    return 6-best_string, best_fret

In [ ]:

def analyze_midi_guitar(midi_data):
    notes = midi_data.instruments[0].notes
    sorted_notes = sorted(notes, key=lambda x: x.start)

    string_names = ['E', 'A', 'D', 'G', 'B', 'e']

    tab_data = []
    for note in sorted_notes:
        string, fret = midi_pitch_to_guitar_tab(note.pitch)
        if string != 1:  # Skip notes on the highest string (e)
            new_note = ({
                'pitch': note.pitch,
                'string': string,
                'fret': fret,
                'string_name': string_names[6-string],
                'start': note.start,
                'end': note.end,
                'duration': note.end - note.start,
                'velocity': note.velocity
            })

            # Check if this note can be merged with the previous one
            if tab_data and tab_data[-1]['pitch'] == new_note['pitch'] and \
               tab_data[-1]['string'] == new_note['string'] and \
               tab_data[-1]['fret'] == new_note['fret'] and \
               abs(tab_data[-1]['end'] - new_note['start']) < 0.1:  # Adjust this threshold as needed
                # Merge with previous note
                tab_data[-1]['end'] = new_note['end']
                tab_data[-1]['duration'] = tab_data[-1]['end'] - tab_data[-1]['start']
                tab_data[-1]['velocity'] = max(tab_data[-1]['velocity'], new_note['velocity'])
            else:
                # Add as a new note
                tab_data.append(new_note)

    return tab_data

In [ ]:

# Assuming midi_data is your MIDI data object
tab_result = analyze_midi_guitar(midi_data)


In [ ]:
# Print the results
for note in tab_result:
    print(f"Pitch: {note['pitch']}, String: {note['string_name']}, String no.: {note['string']}, "
          f"Fret: {note['fret']}, Start: {note['start']:.2f}s, Duration: {note['duration']:.2f}s, "
          f"Velocity: {note['velocity']}")


In [ ]:

len(tab_result)


### Reading chords from notes

In [ ]:

# Identify potential chords (notes played simultaneously)
def identify_chords(tab_data, time_threshold=0.05):
    chords = []
    current_chord = []
    
    for i, note in enumerate(tab_data):
        if not current_chord or note['start'] - current_chord[-1]['start'] <= time_threshold:
            current_chord.append(note)
        else:
            if len(current_chord) > 1:
                chords.append(current_chord)
            current_chord = [note]
        
        if i == len(tab_data) - 1 and len(current_chord) > 1:
            chords.append(current_chord)
    
    return chords

chords = identify_chords(tab_result)

print("\nPotential Chords:")
for i, chord in enumerate(chords):
    print(f"Chord {i + 1}:")
    for note in chord:
        print(f"  String: {note['string_name']}, Fret: {note['fret']}")
    print()


### Saving output as midi file

In [ ]:
def save_midi_data(midi_data, filename):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set tempo
    tempo = mido.bpm2tempo(120)  # Assuming 120 BPM from the original data
    track.append(MetaMessage('set_tempo', tempo=tempo))

    # Add program change for the instrument
    track.append(Message('program_change', program=4, time=0))

    # Add note events
    for note in midi_data.instruments[0].notes:
        # Note on
        track.append(Message('note_on', note=note.pitch, velocity=note.velocity, 
                             time=int(note.start * 1000)))  # Convert to milliseconds
        # Note off
        track.append(Message('note_off', note=note.pitch, velocity=note.velocity, 
                             time=int((note.end - note.start) * 1000)))  # Duration in milliseconds

    mid.save(filename)


In [ ]:
# Usage
save_midi_data(midi_data, 'output.mid')